# Segments Extraction

In [34]:
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import math
import os
from pydub import AudioSegment
import numpy as np
import json
from tqdm import tqdm
import pandas as pd
import seaborn as sns

In [35]:
DATASET_NAME = 'DATASET_CNN'
DATASET_PATH = f'/home/giacomoschiavo/segments/{DATASET_NAME}'
AUDIO_SOURCE = '/home/giacomoschiavo/Tovanella'

# Species Dict
Create a dictionary to map the scientific name of a species to its common name

In [36]:
from pathlib import Path

all_species = Path("utils/BirdNET_GLOBAL_6K_V2.4_Labels_en_uk.txt").read_text(encoding="utf-8").splitlines()
all_species[:5]

['Abroscopus albogularis_Rufous-faced Warbler',
 'Abroscopus schisticeps_Black-faced Warbler',
 'Abroscopus superciliaris_Yellow-bellied Warbler',
 'Aburria aburri_Wattled Guan',
 'Acanthagenys rufogularis_Spiny-cheeked Honeyeater']

In [37]:
# maps every scientific name to its common name
species_dict = {}
for specie in all_species:
    scientific_name, common_name = specie.split("_")    # <Abroscopus albogularis>_<Rufous-faced Warbler>
    species_dict[scientific_name] = common_name

species_dict["Abroscopus albogularis"]

'Rufous-faced Warbler'

In [38]:
# export species_dict to json
# with open('utils/species_dict_map.json', 'w') as f:
#     json.dump(species_dict, f)

# Category and Audio info files
This files are precious as they represent the information about every category and every audio 

In [39]:
# extract annotations from the given file
bird_tags = scipy.io.loadmat('Bird_tags_Train.mat')["Bird_tags"] 
# visualize an example, showing all the properties
for i, prop in enumerate(bird_tags[12][0][0][0]):
    print(i, prop)

0 ['Fringilla_coelebs']
1 ['20190621_030000.WAV']
2 [[ 6.08474576  1.61016949  1.61016949  6.08474576 42.61703208 45.50069122]]
3 [[42.61703208  6.08474576]
 [42.61703208  1.61016949]
 [45.50069122  1.61016949]
 [45.50069122  6.08474576]
 [42.61703208  6.08474576]]
4 [[2]]


+ **Category Info**: group segments by its category
+ **Audio Info**: group segments by its original audio

In [40]:
def get_audio_category_info(bird_tags_filepath, audio_source_path, out_category_info="utils/category_info.json", out_audio_info="utils/audio_info.json"):
    bird_tags = scipy.io.loadmat(bird_tags_filepath)["Bird_tags"]
    category_info = {}      # detections grouped by category
    audio_info = {}         # detections grouped by audio
    for elem in bird_tags:
        # get <scientific name>_<common name> format 
        tag = elem[0][0][0][0][0]
        scientific_name = tag.replace("_", " ")                 # Fringilla coelebs -> Fringilla coelebs
        common_name = species_dict.get(scientific_name, "")     # Fringilla coelebs -> Common Chaffinch
        label = "_".join([scientific_name, common_name])        # Fringilla coelebs_Common Chaffinch
        # get source file
        file_name = elem[0][0][0][1][0]
        file_path = os.path.join(audio_source_path, file_name)
        # duration calculation
        start_time, end_time = np.array(elem[0][0][0][2]).flatten()[-2:]
        duration = end_time - start_time
        # do not store info if file do not exist
        if not os.path.exists(file_path):   
            continue
        # save in dictionaries
        if label not in category_info:
            category_info[label] = []
        if file_name not in audio_info:
            audio_info[file_name] = []
        category_info[label].append({ "file_name": file_name, "start_time": start_time, "duration": duration, "label": label  })
        audio_info[file_name].append({ "scientific_name": scientific_name, "common_name": common_name, "start_time": start_time, "duration": duration, "label": label })
    return category_info, audio_info


In [41]:
# category_info_test, audio_info_test = get_audio_category_info("Bird_tags_Test.mat")

# with open("utils/category_info_test.json", "w") as f:
#     json.dump(category_info_test, f)
# with open("utils/audio_info_test.json", "w") as f:
#     json.dump(audio_info_test, f)

In [42]:
# category_info, audio_info = get_audio_category_info("Bird_tags_Train.mat")
category_info, audio_info = get_audio_category_info("Bird_tags_Train.mat", AUDIO_SOURCE)

# with open("utils/category_info.json", "w") as f:
#     json.dump(category_info, f)
# with open("utils/audio_info.json", "w") as f:
#     json.dump(audio_info, f)

In [43]:
# creates species list
species_list = category_info.keys()

# Segments Creation
Creates all the segments listed in Category Info

In [44]:
def generate_segments(target_path, audio_source_path, category_info, generate=False): 
    true_segments = {}
    species_list = category_info.keys()
    categories = list(species_list)
    for j, category in enumerate(categories):
        all_category_audio = category_info[category]  
        total_length = len(all_category_audio)
        print(f"Loading {category} category... {j}/{len(categories)}")
        for i in tqdm(range(total_length)):
            audio_path = all_category_audio[i]["file_name"] 
            file_name, wav = audio_path.split('.')
            start_time = all_category_audio[i]["start_time"]
            duration = all_category_audio[i]["duration"]
            starting_points = []
            if duration < 1.5:    # ignore
                continue
            n_segments = int(duration // 1.5) - 1
            if n_segments > 0:
                starting_points = [start_time + i*3 for i in range(n_segments)]
            last_segment_duration = duration - n_segments * 3
            if last_segment_duration >= 1.5:
                starting_points.append(start_time + n_segments * 3 + 1.5)
            back_segment_start = start_time - 1.5
            if back_segment_start >= 0:
                starting_points.append(back_segment_start)
            front_segment_end = start_time + duration + 1.5
            if front_segment_end < 10*60:
                starting_points.append(front_segment_end - 3)

            # print(start_times, start_segms)
            if audio_path not in true_segments:
                true_segments[audio_path] = {}
            for start_time in starting_points:
                segm = int(start_time // 3)
                if segm not in true_segments[audio_path]:
                    true_segments[audio_path][segm] = []
                true_segments[audio_path][segm].extend([category])

            if generate:
                os.makedirs(target_path, exist_ok=True)
                audio = AudioSegment.from_file(
                    os.path.join(audio_source_path, audio_path), 
                    format="wav"
                )
                for i, start_time in enumerate(starting_points):
                    segm_id = int(start_time // 3)
                    export_path = os.path.join(
                        target_path,
                        category, 
                        f"{file_name}_{segm_id}.wav"
                    )
                    if os.path.exists(export_path):
                        continue
                    segment = audio[start_time*1000:start_time*1000 + 3000]
                    os.makedirs(os.path.join(target_path, category), exist_ok=True)
                    segment.export(export_path, format="wav")
    return true_segments

In [49]:
# 9 minuti
true_segments = generate_segments(target_path=f"{DATASET_PATH}/train", 
                  audio_source_path=AUDIO_SOURCE, 
                  category_info=category_info, 
                  generate=True)

Loading Wind_ category... 0/31


100%|██████████| 92/92 [00:06<00:00, 13.44it/s]


Loading Regulus ignicapilla_Common Firecrest category... 1/31


100%|██████████| 1297/1297 [00:45<00:00, 28.37it/s]


Loading Sylvia atricapilla_Eurasian Blackcap category... 2/31


...

100%|██████████| 5/5 [00:00<00:00, 79137.81it/s]


Loading Poecile palustris_Marsh Tit category... 29/31


100%|██████████| 1/1 [00:00<00:00, 23.67it/s]


Loading unknown_ category... 30/31


100%|██████████| 3/3 [00:00<00:00, 71.29it/s]


In [ ]:
# dataset_name = "NEW_DATASET"
# true_segments_test = generate_segments(target_path=f"E:/Giacomo/Tovanella/{dataset_name}/final_test", 
#                   audio_source_path="E:/Giacomo/Tovanella/Tovanella", 
#                   category_info=category_info_test, 
#                   generate=False)

Loading Vegetation_ category... 0/22


100%|██████████| 21/21 [00:00<00:00, 20951.57it/s]


Loading Troglodytes troglodytes_Eurasian Wren category... 1/22


100%|██████████| 25/25 [00:00<00:00, 25103.57it/s]


Loading Fringilla coelebs_Common Chaffinch category... 2/22


...

100%|██████████| 11/11 [00:00<00:00, 11016.56it/s]


Loading Certhia familiaris_Eurasian Treecreeper category... 20/22


100%|██████████| 1/1 [00:00<00:00, 1011.89it/s]


Loading Periparus ater_Coal Tit category... 21/22


100%|██████████| 13/13 [00:00<00:00, 12982.37it/s]


In [50]:
# remove duplicate labels in segments
for audio in true_segments:
    for segm in true_segments[audio]:
        true_segments[audio][segm] = list(set(true_segments[audio][segm]))

In [51]:
os.makedirs(f'utils/{DATASET_NAME}', exist_ok=True)
with open(f'utils/{DATASET_NAME}/true_segments.json', 'w') as f:
    json.dump(true_segments, f)

In [ ]:
# with open(f'utils/{DATASET_NAME}/true_segments_test.json', 'w') as f:
#     json.dump(true_segments_test, f)

# WABAD Dataset Integration (to be moved)
Same procedure as above but for another dataset, requires a different treatment

In [47]:
# import species_dict_map
with open('utils/species_dict_map.json') as json_file:
    species_dict_map = json.load(json_file)

In [48]:
allowed_species_name = [species.split("_")[0] for species in species_list]

In [49]:
import csv
def convert_to_json(input_file):
    audio_info = {}
    category_info = {}
    for txt_file in os.listdir(input_file):
        complete_path = os.path.join(input_file, txt_file)
        with open(complete_path, newline='', encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile, delimiter='\t') 
            for row in reader:
                if row["Species"] not in allowed_species_name or "End Time (s)" not in row:
                    continue
                file_name = txt_file.replace(".txt", ".WAV")
                start_time = float(row["Begin Time (s)"])
                end_time = float(row["End Time (s)"])
                duration = end_time - start_time
                common_name = species_dict_map[row["Species"]]
                label = f"{row['Species']}_{common_name}"

                audio_info_entry = {
                    "scientific_name": row["Species"],
                    "common_name": common_name, 
                    "start_time": start_time,
                    "duration": duration,
                    "label": label
                }
                category_info_entry = {
                    "file_name": file_name,
                    "start_time": start_time,
                    "duration": duration,
                    "label": label
                }
                
                if file_name not in audio_info:
                    audio_info[file_name] = []
                if label not in category_info:
                    category_info[label] = []
                audio_info[file_name].append(audio_info_entry)
                category_info[label].append(category_info_entry)
    return audio_info, category_info

In [50]:
wabad_path = "E:/Giacomo/Tovanella/WABAD"
audio_info_ext = {}
category_info_ext = {}
for folder in os.listdir(wabad_path):
    if not os.path.isdir(os.path.join(wabad_path, folder)):
        continue
    annotations = os.path.join(wabad_path, folder, folder, "Raven Pro annotations")
    audio_info_update, category_info_update = convert_to_json(annotations)
    for audio in audio_info_update.keys():
        if audio not in audio_info_ext:
            audio_info_ext[audio] = []
        audio_info_ext[audio].extend(audio_info_update[audio])
    for category in category_info_update.keys():
        if category not in category_info_ext:
            category_info_ext[category] = []
        category_info_ext[category].extend(category_info_update[category])    

# with open("utils/audio_info_ext.json", 'w', encoding='utf-8') as jsonfile:
#     json.dump(audio_info_ext, jsonfile)

# with open("utils/category_info_ext.json", 'w', encoding='utf-8') as jsonfile:
#     json.dump(category_info_ext, jsonfile)

In [51]:
true_segments_ext = generate_segments(target_path="E:/Giacomo/Tovanella/wabad_segments", 
                  audio_source_path="E:/Giacomo/Tovanella/WABAD/", 
                  category_info=category_info_ext, 
                  generate=False)

Loading Phylloscopus collybita_Common Chiffchaff category... 0/22


100%|██████████| 500/500 [00:00<00:00, 47968.89it/s]


Loading Phylloscopus trochilus_Willow Warbler category... 1/22


100%|██████████| 418/418 [00:00<00:00, 139243.83it/s]


Loading Dendrocopos major_Great Spotted Woodpecker category... 2/22


...

100%|██████████| 229/229 [00:00<?, ?it/s]


Loading Loxia curvirostra_Common Crossbill category... 20/22


100%|██████████| 25/25 [00:00<?, ?it/s]


Loading Muscicapa striata_Spotted Flycatcher category... 21/22


100%|██████████| 117/117 [00:00<?, ?it/s]


In [ ]:
# with open("utils/true_segments_ext.json", "w") as f:
#     json.dump(true_segments_ext, f)